# **Forelesning 5 - Bildeklassifisering**
I denne forelesningen skal vi se nærmere på bruk av *bilder* som data, og hvordan vi kan gjøre binær bildeklassifikasjon. Denne notatboken går i hånd med kompendiumet, og fysiske forelesning om tema *Convolutional neural networks*.

## Maskinlæring i samfunnsøkonomi
Dette kurset har som formål å lære dere verktøy fra maskinlæring. Vi har diskutert lineær regresjon, logistisk regresjon, beslutningstrær, random forests, nevrale nettverk og ulike versjoner av sistnevnte.

Maskinlæring har mange bruksområder innen samfunnsøkonomi, særlig fordi den kan analysere store mengder data, finne mønstre og gi oss prediksjoner.  

## 1. Makroøkonomiske prognoser  
- Forutsi BNP-vekst, inflasjon og arbeidsledighet ved å analysere store datasett med økonomiske indikatorer.  
- Sentralbanker kan bruke maskinlæring for å tilpasse sine økonomiske avgjørelser.  

## 2. Arbeidsmarked og lønnsdynamikk  
- Analysere jobbmarkedet og forutsi arbeidsledighet basert på jobbutlysninger, sosiale medier og økonomiske data.  
- Kartlegge hvordan automatisering påvirker sysselsetting i ulike sektorer.  

## 3. Finansiell stabilitet og risikovurdering  
- Oppdage økonomiske bobler ved å analysere finansmarkeder i sanntid.  
- Identifisere systemiske risikoer i banker og finansinstitusjoner.  

## 4. Skatteanalyse og svindeloppdagelse  
- Forutsi skatteinntekter og optimalisere skattesystemet.  
- Bruke algoritmer til å avdekke skattesvindel ved å analysere avvik i regnskapsdata.  

## 5. Modellering av forbrukeratferd  
- Analysere kjøpsmønstre og hvordan økonomiske faktorer påvirker forbruk.  
- Utvikle mer treffsikre modeller for prisdynamikk og etterspørsel.  

## 6. Evaluering av offentlige tiltak  
- Måle effekten av ulike økonomiske politikktiltak (for eksempel subsidier eller velferdsordninger) ved å sammenligne store datasett før og etter implementering.  

## 7. Handels- og investeringsanalyse  
- Forutsi effekten av tollsatser, handelsavtaler og geopolitikk på økonomien.  
- Analysere investeringstrender ved hjelp av alternative data, som satellittbilder og sentimentanalyse fra nyheter.  

Maskinlæring kan altså gi økonomer et bedre verktøy for å forstå komplekse systemer og ta mer informerte beslutninger.  

## Bildeklassifisering i samfunnsøkonomi
Bildeklassifisering med **Convolutional Neural Networks (CNN)** kan brukes i samfunnsøkonomi for å trekke innsikt fra visuelle data.  

## 1. Økonomisk overvåking med satellittbilder  
- **Måling av økonomisk aktivitet**: CNN kan analysere nattlys-satellittbilder for å estimere økonomisk vekst i ulike regioner.  
- **Handelsvolum og forsyningskjeder**: Overvåke aktiviteten i havner og transportknutepunkter for å vurdere internasjonal handel.  
- **Jordbruk og ressursforvaltning**: Predikere matproduksjon og økonomiske effekter av avlingssvikt ved analyse av satellittbilder.  

## 2. Forbruksanalyse og detaljhandel  
- **Kundeatferd i butikker**: Overvåke kundestrømmer i fysiske butikker for å optimalisere produktplassering og bemanning.  
- **Kø- og trafikkmønstre**: CNN kan analysere trafikk- og parkeringsdata for å vurdere økonomisk aktivitet i ulike områder.  

## 3. Arbeidsmarked og automatisering  
- **Yrkesklassifisering via bilder**: CNN kan analysere bilder fra jobbsøkere for å kartlegge yrkesroller basert på arbeidsklær, verktøy og omgivelser.  

## 4. Økonomisk krisehåndtering  
- **Naturkatastrofer og økonomiske tap**: CNN kan evaluere skader fra flom, branner eller jordskjelv for å estimere økonomiske konsekvenser.  
- **Flyktningstrømmer og migrasjon**: Bildeanalyse kan brukes for å overvåke bevegelsesmønstre ved humanitære kriser.  

Ved å kombinere CNN med økonomiske data kan vi få dypere innsikt i hvordan visuelle signaler påvirker økonomiske beslutninger og trender.  
___

# **Eksempel - klassifisering av skip**
I denne notatboken skal vi se hvordan vi kan implementere CNN til å klassifisere om en bilde av et skip er *militært skip* eller *sivilt skip*.

## Installere Dependencies og libraries
Før vi kan begynne koden, trenger vi å installere noe bibliotek.

In [ ]:
!pip install tensorflow
!pip install opencv-python # Computer vision modul
!pip install matplotlib
!pip list

### Tensorflow
Vi importerer TensorFlow for å kunne gjøre koden våre.

In [ ]:
import tensorflow as tf
import os

Her kan vi sjekke hvor mange GPU'er vi har tilgjengelig. Dette kan vi endre i Google Colab ved å endre i "Kjøring" --> "Endre kjøringstype".

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
gpus

Neste linjer med kode er satt for å forhindre minneproblemer når vi kjører store maskinlæringsmodeller.

In [ ]:
# Avoid OOM errors (Out of memory) by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
tf.config.list_physical_devices('GPU')

## Data
Vi skal her se på bilder av militære (0) og sivile (1) båter/skip.
Vi har masse data-eksempler i en mappe, og må *cleane* den for eventuelle dårlige bilder.

### Hvorfor
Et viktig prinsipp i maskinlæring er *shit in, shit out*. Da mener vi at hvis vi har dårlig data, vil modellen vår bli dårlig, noe som vil gi dårlig output av modellen.

In [ ]:
# Importerer noen essensielle bibliotek
import cv2                        # For fargekonvertering, og computer-vision
import imghdr                     # Modul som bestemmer hvilke type bilde som er i en fil - brukes for å cleane data.
import matplotlib.pyplot as plt   # For plotting, vise bilder etc.

# Hvordan hente data:
I denne forelesningen vil vi gjennomgå hvordan vi lager en binær-bildeklassifiseringsmodell for å detektere om hva vi ser på bilde er **MILITÆR SKIP** eller **SIVILE SKIP**. Denne koden kan i prinsippet brukes for mange andre binære, bildeklassifiseringsmodeller - men hvor man kanskje må endre visse ting som nettverksarkitekturen.

### Hente bilder fra internett
1. Gå inn på Google Chrome, og last ned en "Utvidelse" ("Extension") som heter "Download All Images".
2. Gå til Google, og skriv inn "army ships".
3. Trykk på "Utvidelser" og trykk "Download All Images".
4. Nå vil alle bildene lastes ned lokalt på PC/Mac'en din.
5. Repeter steg 2 - 4, men ved å søke "Civilian ships".
6. Lag en mappe som heter "data-ships", som inneholder to mapper: "army" og "non-army".
7. Last opp "data-ships" mappen din i "Min disk" ("MyDrive") på Google drive.
8. Nå er du klar for å kjøre koden under.


Tips: Før du laster opp i Google Drive, kan det være lurt å fjerne de minste filene. F.eks. de som er under 10 kB, da de ofte er veldig små og kan by på problemer.

# Last data inn i Google Colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Define the path to your folder
data_dir = '/content/drive/MyDrive/data-ships'
data_dir

## Sjekker hvilke mapper som er inne i vårt *data_dir*

In [ ]:
os.listdir(data_dir)

In [ ]:
# Får en liste av sub-directories (foldere) som er en del av vår data_dir
folders = [f for f in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, f))]

# Teller files i hver mappe/foldere
file_counts = {folder: len(os.listdir(os.path.join(data_dir, folder))) for folder in folders}

# Print results
for folder, count in file_counts.items():
    print(f"Folder '{folder}' contains {count} files.")

## Her kan vi få opp alle filnavnene under mappen 'army'

In [ ]:
os.listdir(os.path.join(data_dir, 'army'))

In [ ]:
image_exts = ['jpeg','jpg', 'bmp', 'png']

In [ ]:
img = cv2.imread(os.path.join(data_dir, 'army', '1647297548254.jpg'))

In [ ]:
img_2 = cv2.imread(os.path.join(data_dir, 'army',  'HMS Queen Elizabeth -MoD-.jpg'))

In [ ]:
print(img.shape)
print(img_2.shape)

In [ ]:
# OPENCV leser bilder som BGR, matplotlib forventer RGB
plt.imshow(img)
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB)) # Konverterer BGR til RGB
plt.show()

In [ ]:
### Funksjon for å fjerne potensielle kjipe filer, som vi ikke trenger ###

for image_class in os.listdir(data_dir):
    for image in os.listdir(os.path.join(data_dir, image_class)):
        image_path = os.path.join(data_dir, image_class, image)
        try:
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in image_exts:
                print('Image not in ext list {}'.format(image_path))
                os.remove(image_path)
        except Exception as e:
            print('Issue with image {}'.format(image_path))
            #os.remove(image_path)

## Laste inn data'en
Sjekk ut `tf.data.Dataset`, som vi kommer til å bruke mye her. Tensorflow dokumentasjon, til dem som skal gjøre bildeklassifisering i prosjektet finnes [her](https://www.tensorflow.org/api_docs/python/tf/data/Dataset)!

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
# Dette her er gull - tf.keras.utils.image_dataset_from_directory er fantastisk for bildeklassifisering
data = tf.keras.utils.image_dataset_from_directory(data_dir)

In [ ]:
# Kjøre denne kodeblokken for å se hva vi kan konfigurere!
tf.keras.utils.image_dataset_from_directory??

In [ ]:
# Shuffle (stokking av dataene)
data_iterator = data.as_numpy_iterator()

In [ ]:
# Får en ny batch fra vår "iterator" - "vi henter en batch"
batch = data_iterator.next()

# Denne batch'en består av bilder (i numpy format), og labels/merkinger (0 or 1, army/not-army)

In [ ]:
len(batch)

In [ ]:
# Se på shape til ett bilde
batch[0].shape

In [ ]:
# Aksessere bildene (images)
batch[0]

# Aksessere labels
batch[1]

In [ ]:
# 0 = Army (militære) boats
# 1 = Civilian (sivile) boats

fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx, img in enumerate(batch[0][:4]):
    #ax[idx].imshow(img)
    ax[idx].imshow(img.astype(int))
    ax[idx].title.set_text(batch[1][idx])

## Klassifisering
Vi har tidligere snakket om forskjellen mellom *regresjon* og *klassifikasjon*, her gjør vi altså sistnevnte.

Som vi kan se over, så representerer '0' militære skip og '1' ikke-militære. Det som er fint med koden vi presenterer her er at dette kan i prinsippet trenes for all mulig slags use-case'r med binær bildeklassifisering.

In [ ]:
# Selve bildene er hvor, key = 0
batch[0]

In [ ]:
# Labels er i key = 1 (altså, om bildene av skipene er sivile eller militære)
batch[1]

In [ ]:
# Siden batch[0] er bildene våre, vil batch[0].shape gi (#number of images, 256, 256, 3) ---> 3 kanaler=fargebilde.
# Så hvis vi ser på batch[0].min() og batch[0].max() - vil de returnere henholdsvis 0.0 and 255.0
batch[0].shape

In [ ]:
batch[0].min()

In [ ]:
batch[0].max()

Husk når vi diskuterte *rank-3 tensorer*, da det er tre matriser *stakket* over hverandre. Derfor snakker vi ofte om RGB, da det er et *lag*/matrise per farge.

![Bilde](https://miro.medium.com/v2/resize:fit:1400/1*8pX8Zt2PvIswXv3JY5_AWg.png)


### Når vi bruker dyp-læringsmodeller, bør verdiene vi jobber med (ideelt sett) være så små som mulig!
Dette gjøres for de da går *optimaliseringen* MYE fortere. Da kan vi normalisere, det gjør vi ved å dele alle bilde-verdiene på $255$, kall disse $x_i$, og da vil vi få normaliserte verdier - $x_i* \in[0, 1]$.

Dette kalles i *rescaling* eller *min-max normalisering* og kan uttrykkes matematisk:
$$x_{i*}=\frac{x_i-min(\mathbf{x})}{max(\mathbf{x})-min(\mathbf{x})}$$

I dette tilfellet er $min(\mathbf{x})=0$ og $max(\mathbf{x})=255$.


Uansett, det viktige her er at da vil optimalisering gjennom 'ADAM' eller 'Stochastic Gradient Decent' (som diskutert i forelesning 2), vil gå betydelig fortere.

In [ ]:
# Her skalerer vi dataen vår til å være mellom 0 og 1.
# data.map funker bra for transformasjon av data.
# lambda funksjonen lar oss skalerer bildet, altså x - og lar y være som den er.
data = data.map(lambda x,y: (x/255, y))

I koden over vil skalere bildene $x$ med å dele på $255$, slik at hver piksel består av en verdi mellom $0$ og $255$. Men $y$ forblir den samme, som er $0$ eller $1$, basert på om bilde er sivilt eller et militært skip.

In [ ]:
# Her ser vi at dataene er normalisert, altså mellom 0 og 1.
data.as_numpy_iterator().next()

## Splite data inn i trening, validering og test sett.

In [ ]:
len(data)

In [ ]:
### Dette splitter data inn i trening, validering og test. ###

train_size = int(len(data)*.7)
val_size = int(len(data)*.2)
test_size = int(len(data)*.1)

In [ ]:
# Så her får vi opp hvor mange batcher vi har i treningsdataene, hvor batch_size ble satt lenger oppe
train_size

In [ ]:
# Så her får vi opp hvor mange batcher vi har i valideringsdataene, hvor batch_size ble satt lenger oppe
val_size

In [ ]:
test_size

In [ ]:
### Her bruker jeg train og skip metoder fra tensorflow for å sortere data. Her er det viktig at dataene allered er shuffled. ###

train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)

# Nå kan vi bygge vår bilde-klassifiseringsmodell!

In [ ]:
train

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

In [ ]:
# Vi lager en sekvensiell modell, og legger på lag ved hjelp av 'model.add'-metoden.
model = Sequential()

In [ ]:
### Så her bygger vi vår faktiske modell ###

# There are the Convolutional blocks, one flatten layer and two dense layers.

# This adds a 2D Convolutional layers - sequantially. The input is a color-image.
# 16 filters, 3x3 pixels in size, and a stride of 1 pixel. This is ML architecture.
# ReLu activation function, outputs will be passed through a ReLu function! Will
# capture non-linear patterns in our data, which is great and a huge application of ML:-)
# image shape is 256x256 images, and three layers/channels deep (because its color images).
model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(256,256,3)))
model.add(MaxPooling2D())

# 32 filter.
model.add(Conv2D(32, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())

# 16 filters
model.add(Conv2D(16, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())

# Here we flatten the data
model.add(Flatten())

# Now we have the flatten layer connected to 256 neurons.
model.add(Dense(512, activation='relu'))

# Single dense layer, and use sigmoid activation function to return 0 or 1 - using 0.5 as a threshold.
model.add(Dense(1, activation='sigmoid'))

In [ ]:
# Her bruker vi opitimzer='adam', men den kan endres på!
# Loss er 'binary cross entropy', og vi får tilbake accuracy (fra en conf. matrix).
model.compile('adam', loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy'])

In [ ]:
# Here we get the summary of our NN is doing!
# So max_pooling2D dont add any parameters, as you can see.
# 30x30x16 = 14400 - when we flatten our data.
# 257 in the end, is 256 + the bias term.
model.summary()

## Nå kan vi begynne å *trene*

In [ ]:
logdir='logs'

In [ ]:
# Nice to save our model, and log our training.
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
# Se på training data
train

In [ ]:
# model.fit is for training our model. Epoch, one epoch is one run over our entire
# training data. And we also going to iterate through our validation data.
# and we load our callbacks, to look at our training process later.
hist = model.fit(train, epochs=20, batch_size=1, validation_data=val, callbacks=[tensorboard_callback])

## Vi plotter hvordan treningen går

In [ ]:
# Se på vår performance for training og validation.
hist.history

In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.show()

## Nå kan vi evalurer modellen vår
Dette gjør vi altså ved å teste modellen på det **u-sette** test settet!

In [ ]:
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy

In [ ]:
pre = Precision()
re = Recall()
acc = BinaryAccuracy()

In [ ]:
for batch in test.as_numpy_iterator():
    X, y = batch
    yhat = model.predict(X)
    pre.update_state(y, yhat)
    re.update_state(y, yhat)
    acc.update_state(y, yhat)

In [ ]:
print(pre.result(), re.result(), acc.result())

## La oss ta en tilfeldig test av et nytt bilde

In [ ]:
import cv2

img = cv2.imread(os.path.join(data_dir, 'army', '1647297548254.jpg'))
#plt.imshow(img)
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB)) # Convert BGR to RGB
plt.show()

In [ ]:
resize = tf.image.resize(img, (256,256))
plt.imshow(resize.numpy().astype(int))
plt.show()

In [ ]:
# Here we normalize the data by dividing the 'resize' image by 255, to be in the scale [0, 1] --> because then ML stuff goes easier and faster.
yhat = model.predict(np.expand_dims(resize/255, 0))

In [ ]:
yhat

In [ ]:
if yhat > 0.5:
    print(f'Predicted class is non-Army')
else:
    print(f'Predicted class is Army')

## Eksempelet fra sist gang - MNIST

[Se på denne!](https://adamharley.com/nn_vis/cnn/2d.html)

# **Kode eksempel - Convolutional Neural Networks**
Her er et kode-eksempel som bruker et spesielt type nettverk, CNN, dette vil vi ser mer på i forelesning 5.

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

In [ ]:
# Laster inn MNIST-datasettet (håndskrevne siffer).
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normaliserer bildene til verdier i området [0, 1] ved å dele på 255.
# Dette gjør at pikselverdiene, som opprinnelig er i området [0, 255], skaleres til flyttall mellom 0 og 1.
x_train = x_train / 255
x_test = x_test / 255

In [ ]:
# Laster inn MNIST-datasettet (håndskrevne siffer).
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normaliserer bildene til verdier i området [0, 1] ved å dele på 255.
# Dette gjør at pikselverdiene, som opprinnelig er i området [0, 255], skaleres til flyttall mellom 0 og 1.
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# Endrer formen på bildene fra (num_samples, 28, 28) til (num_samples, 28, 28, 1).
# Den ekstra dimensjonen (1) representerer antall kanaler (her gråskala), hadde den vært satt til (3) har vi fargebilder
# da de har tre fargekanaler RGB (Rød, Grønn, Blå).
x_train = x_train.reshape((-1, 28, 28, 1))
x_test = x_test.reshape((-1, 28, 28, 1))

#Obs: Den første "-1" i reshape() lar NumPy automatisk beregne antall eksempler (num_samples).


# One-hot encoder etikettene (klassene) slik at hver etikett blir representert som en vektor med 10 elementer.
# Eksempel: Hvis en etikett er 3, blir den representert som [0, 0, 0, 1, 0, 0, 0, 0, 0, 0].
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

Når du setter den første dimensjonen til `-1`, vil `reshape()`-funksjonen fra NumPy automatisk finne ut hvor mange bilder det er, basert på størrelsen på resten av dimensjonene $(28, 28, 1)$. På denne måten slipper du å manuelt spesifisere antall bilder `num_samples()` (Men, her vet vi at det er 60.000 bilder i treningsdataene - fra MNIST).

In [ ]:
# Importere nødvendige biblioteker fra TensorFlow
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout

# Lager en sekvensiell modell
model = Sequential([
    # Første konvolusjonslag
    # Bruker 32 filtre av størrelse (3, 3), aktivert med ReLU, og forventer en inputform av (28, 28, 1) - altså våre gråskalabilder.
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),

    # Maksimal pooling-lag som reduserer dimensjonaliteten til funksjonskartene.
    # Bruker en pooling størrelse på (2, 2), noe som halverer både bredde og høyde.
    tf.keras.layers.MaxPooling2D((2, 2)),

    # Dropout-lag for å forhindre overfitting ved tilfeldig å sette 25% av nevronene til null under trening.
    Dropout(0.25),

    # Andre konvolusjonslag med 64 filtre, fortsatt med ReLU-aktivering.
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),

    # Ny maksimal pooling-lag for ytterligere dimensjonsreduksjon.
    tf.keras.layers.MaxPooling2D((2, 2)),

    # Ytterligere dropout-lag for å forhindre overfitting, denne gangen med 25%.
    Dropout(0.25),

    # Flatten-lag som konverterer de 2D-funksjonskartene til 1D-vektorer.
    tf.keras.layers.Flatten(),

    # Første Dense-lag med 128 nevroner og ReLU-aktivering, som fullstendig kobler til forrige lag.
    Dense(128, activation='relu'),

    # Dropout-lag for å hindre overfitting i dette laget også, nå med 50%.
    Dropout(0.5),

    # Utgangslag med 10 nevroner som representerer de 10 klassene, aktivert med softmax for å få sannsynligheter.
    Dense(10, activation='softmax')
])

# Modellstrukturen er nå definert. Den kan kompileres og trenes på datasett etterpå.

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(x_train, y_train, epochs=10)

In [ ]:
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f'Test accuracy: {test_accuracy}: Test loss: {test_loss}')

## Her ser vi at vi har fått en test accuracy på over $99$%!
Her har vi brukt en annen form for nevrale nettverk, kalt *Convolutional Neural Networks* - som vi vil se mer på i forelesning 5!